In [1]:
from joblib import dump, load
import pandas as pd
from sklearn.feature_extraction import FeatureHasher
import numpy as np
from sklearn.metrics import log_loss
from sklearn import decomposition

In [2]:
column_names = ['interger1','interger2','interger3','interger4','interger5','interger6',
                'interger7','interger8','interger9','interger10','interger11','interger12','interger13',
                'categorical1','categorical2','categorical3','categorical4','categorical5','categorical6',
                'categorical7','categorical8','categorical9','categorical10','categorical11','categorical12',
                'categorical13','categorical14','categorical15','categorical16','categorical17','categorical18',
                'categorical19','categorical20','categorical21','categorical22','categorical23','categorical24',
                'categorical25','categorical26']

In [3]:
test = pd.read_csv('/home/bahbbc/workspace/display-ads-challenge/dac/test.txt', sep='\t',
            names=column_names, chunksize=327433, engine='python')

In [4]:
clt = load('regularized-pca-logistic.pkl')
scaler = load('integer-pca-scaler.pkl')
svd = load('svd-reduction.pkl')

In [5]:
integer_cols = ['interger2', 'interger8', 'interger5','interger11', 'interger9', 'interger7']
cat_vars = ['categorical8', 'categorical14', 'categorical5', 'categorical12', 'categorical11', 
                'categorical10', 'categorical7', 'categorical1', 'categorical15', 'categorical18', 
                'categorical13', 'categorical16']

In [6]:
feature_hasher = FeatureHasher(n_features=1000)

In [7]:
probs = np.array([0,0])
for chunk in test:
    #pre-processing
    X = chunk[integer_cols + cat_vars]
    X.update(X[integer_cols].fillna(0))
    X.update(X[cat_vars].fillna('NULL'))

    X_integer = scaler.transform(X[integer_cols])
    
    X_cat = feature_hasher.transform(X[cat_vars].to_dict('records')).toarray()
    X_test = np.hstack((X_cat, X_integer))
    
    principal_components = svd.transform(X_test)
    #predict
    prob = clt.predict_proba(principal_components)
    probs = np.vstack((probs, prob))

/home/bahbbc/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  raise_on_error=True)


In [8]:
probs[:10]

array([[ 0.        ,  0.        ],
       [ 0.63079615,  0.36920385],
       [ 0.60781292,  0.39218708],
       [ 0.64802131,  0.35197869],
       [ 0.73183668,  0.26816332],
       [ 0.71200348,  0.28799652],
       [ 0.67283683,  0.32716317],
       [ 0.70638405,  0.29361595],
       [ 0.75138986,  0.24861014],
       [ 0.70858191,  0.29141809]])

In [9]:
len(probs)

6042136

In [10]:
probs[0]                                                          

array([ 0.,  0.])

In [11]:
p = probs[1:6042136]

In [12]:
len(p)

6042135

In [13]:
probs = pd.DataFrame(p, columns=['0','1'])

In [14]:
sample_sub = pd.read_csv('/home/bahbbc/workspace/display-ads-challenge/random_submission.csv', sep=',',)

In [15]:
sample_sub['Predicted'] = probs['1']

In [16]:
sample_sub.head()

,Id,Predicted
0,60000000,0.369204
1,60000001,0.392187
2,60000002,0.351979
3,60000003,0.268163
4,60000004,0.287997


In [17]:
sample_sub.to_csv('pca-probs.csv', index=False)